In [1]:
import os
import rasterio
import numpy as np
from osgeo import gdal, gdalconst
import pandas as pd
from shutil import copy2

# Step0: Resample B08

In [4]:
maindir = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\testingPhase_France\Training_Images\Step0_ResampleB08'
reference = gdal.Open(os.path.join(maindir,'refImg' ,'T31TFJ_20190910T104021_B02_20m.jp2'), gdalconst.GA_ReadOnly)
referenceProj = reference.GetProjection()
referenceTrans = reference.GetGeoTransform()
bandreference = reference.GetRasterBand(1)    
x = reference.RasterXSize 
y = reference.RasterYSize
    
for file in os.listdir(os.path.join(maindir,'input')):
    inputfile = gdal.Open(os.path.join(maindir,'input', file), gdalconst.GA_ReadOnly)
    inputProj = inputfile.GetProjection()
    inputTrans = inputfile.GetGeoTransform()
    
    outputfile = os.path.join(maindir,'output',file[:-8]+'_20m'+'.tiff')
    driver= gdal.GetDriverByName('GTiff')
    output = driver.Create(outputfile,x,y,1,bandreference.DataType)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)

    gdal.ReprojectImage(inputfile,output,inputProj,referenceProj,gdalconst.GRA_Bilinear) # options: nearest neighbor, bilinear, cubic

    del output
    inputFile.FlushCache()
    inputFile = None

In [5]:
# Copy resampled B08 files to another directory

for file in os.listdir(os.path.join(maindir,'output')):
    src = os.path.join(os.path.join(maindir,'output',file))
    dst = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\testingPhase_France\Training_Images\Step1_ApplySCLMask'
    copy2(src, dst)

# Assign -99 to cloud pixels using corresponding SCL files

In [28]:
maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\testingPhase_France\Training_Images\Step1_ApplySCLMask'
dates = ['before','after']

for date in dates:
    subdir1 = os.path.join(maindir1, date)
    scl = rasterio.open(os.path.join(subdir1,os.listdir(subdir1)[-1])).read() # Read the corresponding SCL file
    # Transform the SCL layer into a binary mask 
    vals = np.unique(scl).tolist()
    other_values = vals.copy()
    shadow_cloud_values = [3.0, 8.0, 9.0, 10.0]  # List of shadow and cloud values                              
    for item in shadow_cloud_values:
        if item in other_values:
            other_values.remove(item)    # List of values other than shadow and clouds                                          
    for item in other_values:
        scl[scl==item] = 1
    for item in shadow_cloud_values:
        scl[scl==item] = 0
    for file in os.listdir(subdir1)[:8]:        
        img = rasterio.open(os.path.join(subdir1, file))
        img_arr = img.read() 
        # Apply mask
        cloudMask = np.array(scl)
        new_img = np.multiply(cloudMask, img_arr).astype(np.float32)
        new_img[new_img==0]=-99
        # Save file
        outputdir = os.path.join(maindir1,'output', file[7:30]+'.tiff')
        with rasterio.open(outputdir,'w',driver='Gtiff', width=img.width, height=img.height, 
                      count=1,crs=img.crs,transform=img.transform, dtype=np.float32, nodata=9.96921e+36) as newImg:
            newImg.write(new_img)    
            newImg.close()

# Assign CRS, resample and clip

In [2]:
maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\CopernicusTurbidityLayers\France'

reference = gdal.Open(os.path.join(maindir1, 'c_gls_LWQ100-turbidity-blended-mean_201902110000_CUSTOM_MSI_V1.3.tiff'), gdalconst.GA_ReadOnly)
referenceProj = reference.GetProjection()
referenceTrans = reference.GetGeoTransform()
bandreference = reference.GetRasterBand(1)    
x = reference.RasterXSize 
y = reference.RasterYSize

In [3]:
maindir2 = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\testingPhase_France\Training_Images\Step1_ApplySCLMask'
maindir3 = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\testingPhase_France\Training_Images\Step2_AssignCRS_Resample_Clip'
for file in os.listdir(os.path.join(maindir2, 'output')):  
    inputfile = gdal.Open(os.path.join(maindir2, 'output',file), gdalconst.GA_ReadOnly)    
    inputProj = inputfile.GetProjection()
    inputTrans = inputfile.GetGeoTransform()

    outputfile = os.path.join(maindir3,'resampled_'+file)
    driver= gdal.GetDriverByName('GTiff')
    output = driver.Create(outputfile,x,y,1,bandreference.DataType)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)

    gdal.ReprojectImage(inputfile,output,inputProj,referenceProj,gdalconst.GRA_NearestNeighbour) # choose nearest neighbor to avoid averaging with -99

    del output
    inputfile.FlushCache()
    inputfile = None  

# Apply land mask

In [2]:
maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\Bathymetry\France\binaryMask'
mask_img = rasterio.open(os.path.join(maindir1, 'binaryMask_v3.tiff'))
mask_arr = mask_img.read(1)

maindir2 = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\testingPhase_France\Training_Images\Step2_AssignCRS_Resample_Clip'
maindir3 = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\testingPhase_France\Training_Images\Step3_ApplyElevMask'

names = np.array([]).astype('str')
percent = np.array([]).astype('float')
for file in os.listdir(maindir2):
    img = rasterio.open(os.path.join(maindir2, file))
    arr = img.read(1)
    masked = np.multiply(mask_arr, arr)
    # calculate cloud%
    allNumPixels = np.sum(masked!=0) # nan values
    cloudPixels = np.sum(masked==-99)
    percentage = np.round(100*(cloudPixels/allNumPixels), 2)
    names = np.append(names, file[26:33])
    percent = np.append(percent, percentage)
      
    # Save file
    masked[masked==0]=9.96921e+36 # permanent change for nodata values
    outputdir = os.path.join(maindir3, 'masked_'+file[10:])
    with rasterio.open(outputdir,'w',driver='Gtiff', width=img.width, height=img.height, 
                  count=1,crs=img.crs,transform=img.transform, dtype=np.float32, nodata=9.96921e+36) as newImg:
        newImg.write(masked, indexes=1)    
        newImg.close()

# Save percentage file
df = pd.DataFrame({'Image': names,'Cloud Percentage': percent})
outputdir = r'G:\MScThesis\waterQualityMonitoring\Data\GapFilling\testingPhase_France\Training_Images\summary_cloud_percent.xlsx'
df[7:9].to_excel(outputdir, index=False)